# Part f): Cross-validation as resampling techniques

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
import functions as f
plt.style.use('seaborn-v0_8-whitegrid')
import load_data as ld

x, y, z = ld.load_uniform_data()

# Define the range of polynomial degrees to test
degrees = np.arange(0, 10)

# Define the number of folds
k_folds = 10

# Initialize arrays to store the MSE values
mse_ols = np.zeros_like(degrees, dtype=float)
mse_ridge = np.zeros_like(degrees, dtype=float)
mse_lasso = np.zeros_like(degrees, dtype=float)

# Perform k-fold cross-validation for each degree
for i, degree in enumerate(degrees):
    # Create polynomial features
    poly_features = PolynomialFeatures(degree=degree)
    X = poly_features.fit_transform(np.column_stack((x, y)))

    # Scale and center the data
    X_train, X_test = f.scale_train_test(train = X_train, test = X_test)
    z_train, z_test = f.scale_train_test(train = z_train, test = z_test)

    # Perform k-fold cross-validation for ordinary least squares
    model_ols = LinearRegression(fit_intercept=False)
    mse_ols[i] = -np.mean(cross_val_score(model_ols, X, z, cv=k_folds, scoring='neg_mean_squared_error'))
    
    # Perform k-fold cross-validation for Ridge regression
    mse_temp = []
    for lambda_val in np.arange(0.00001, 1, 5):
        model_ridge = Ridge(alpha=lambda_val)
        mse = -np.mean(cross_val_score(model_ridge, X, z, cv=k_folds, scoring='neg_mean_squared_error'))
        mse_temp.append(mse)
    mse_ridge[i] = np.min(mse_temp)
    
    # Perform k-fold cross-validation for Lasso regression
    mse_temp = []
    for lambda_val in np.arange(0.00001, 1, 5):
        model_lasso = Lasso(alpha=lambda_val)
        mse = -np.mean(cross_val_score(model_lasso, X, z, cv=k_folds, scoring='neg_mean_squared_error'))
        mse_temp.append(mse)
    mse_lasso[i] = np.min(mse_temp)

# Plot the MSE values
plt.figure(figsize=(8, 6))
plt.plot(degrees, mse_ols, label='OLS')
plt.plot(degrees, mse_ridge, label='Ridge')
plt.plot(degrees, mse_lasso, label='Lasso')
plt.xlabel('Polynomial Degree')
plt.ylabel('MSE')
plt.title('MSE for OLS, Ridge, and Lasso')
plt.legend()
plt.show()
